In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasters as rt
from rasters import MultiPoint
from PTJPLSM import PTJPLSM, process_PTJPLSM_table
from PTJPL import load_Topt, load_fAPARmax
from soil_capacity_wilting import load_field_capacity
from soil_capacity_wilting import load_wilting_point
from gedi_canopy_height import load_canopy_height
from ECOv002_calval_tables import load_combined_eco_flux_ec_filtered, load_metadata_ebc_filt, load_calval_table

In [2]:
repo_root = os.path.dirname(os.getcwd())
package_dir = os.path.join(repo_root, 'PTJPLSM')
generated_input_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-SM-inputs.csv")
generated_output_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-SM-outputs.csv")

In [3]:
model_inputs_gdf = load_calval_table()
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,MAP,StartDate,EndDate,LE_count,closure_ratio,geometry,time_UTC,ST_K,ST_C,Ta_C
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,1320.00,10/1/18 05:00,1/1/22 05:00,9576,1.02,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681


In [4]:
# extract array of x and array of y coordinates from data frame geometry
x_coords = model_inputs_gdf.geometry.x
y_coords = model_inputs_gdf.geometry.y
tower_points = MultiPoint(x=x_coords, y=y_coords)
tower_points

MULTIPOINT ((-76.656 35.799), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-75.9038 35.7879), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-90.3004 45.9793), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6

In [5]:
Topt = load_Topt(geometry=tower_points)
model_inputs_gdf["Topt"] = Topt
Topt

array([10.09,  1.39,  1.39, ...,  7.99,  7.99,  7.99], shape=(1065,))

In [6]:
fAPARmax = load_fAPARmax(geometry=tower_points)
model_inputs_gdf["fAPARmax"] = fAPARmax
fAPARmax

array([0.4659, 0.4865, 0.4865, ..., 0.3387, 0.3387, 0.3387], shape=(1065,))

In [ ]:
canopy_height_meters = load_canopy_height(geometry=tower_points)
model_inputs_gdf["canopy_height_meters"] = canopy_height_meters
canopy_height_meters

In [ ]:
field_capacity = load_field_capacity(geometry=tower_points)
model_inputs_gdf["field_capacity"] = field_capacity
field_capacity

In [ ]:
wilting_point = load_wilting_point(geometry=tower_points)
model_inputs_gdf["wilting_point"] = wilting_point
wilting_point

In [ ]:
model_inputs_gdf

In [ ]:
model_inputs_gdf.keys()

In [ ]:
model_inputs_gdf.albedo

In [ ]:
results = process_PTJPLSM_table(model_inputs_gdf)
results

In [ ]:
model_inputs_gdf.to_csv(generated_input_table_filename, index=False)

In [ ]:
results.to_csv(generated_output_table_filename, index=False)